In [1]:
from dotenv import load_dotenv
from ollama import Client
from pydantic import create_model
import inspect, json
from inspect import Parameter

### Custom agent

*Define the function*

In [2]:
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [9]:
abc(2, 3)

-2

In [4]:
def jsonschema(f):
    """
    Generate a JSON schema for the input parameters of the given function.

    Parameters:
        f (FunctionType): The function for which to generate the JSON schema.

    Returns:
        Dict: A dictionary containing the function name, description, and parameters schema.
    """
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
            for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [5]:
abc_json = jsonschema(abc)
abc_json

{'name': 'abc',
 'description': 'Compute abc between two numbers',
 'parameters': {'properties': {'num1': {'title': 'Num1', 'type': 'integer'},
   'num2': {'title': 'Num2', 'type': 'integer'}},
  'required': ['num1', 'num2'],
  'title': 'Input for `abc`',
  'type': 'object'}}

In [6]:
model_name = "llama3-groq-tool-use:latest"

*Ask GPT*

In [7]:
client = Client()

response = client.chat(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "compute abs between 2 and 3"},
    ],
)

In [8]:
response['message']['content']

'The absolute value (abs) between 2 and 3 is 1.'

In [13]:
from typing import List

from langchain_core.tools import tool
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=model_name,
    temperature=0,
).bind_tools([abc_json])

result = llm.invoke(
    "compute abs between 2 and 3"
)
result.tool_calls

[{'name': 'abc',
  'args': {'num1': 2, 'num2': 3},
  'id': 'c2d3b099-2214-4509-9864-aa1e0ae645e6',
  'type': 'tool_call'}]

In [17]:
print(result)

content='' additional_kwargs={} response_metadata={'model': 'llama3-groq-tool-use:latest', 'created_at': '2024-12-23T19:11:57.374347981Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11297903638, 'load_duration': 805162753, 'prompt_eval_count': 162, 'prompt_eval_duration': 7407000000, 'eval_count': 33, 'eval_duration': 3083000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-dc28d9ac-8ce3-45b1-bdfc-256353ca2101-0' tool_calls=[{'name': 'abc', 'args': {'num1': 2, 'num2': 3}, 'id': 'c2d3b099-2214-4509-9864-aa1e0ae645e6', 'type': 'tool_call'}] usage_metadata={'input_tokens': 162, 'output_tokens': 33, 'total_tokens': 195}


### Using Langchain

In [22]:
from langchain_core.tools import tool

@tool
def abc(num1:int, num2:int)->int:
    "Compute abc between two numbers"
    return 2*(num1) - 2*(num2)

In [23]:
abc.description

'Compute abc between two numbers'

In [24]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3-groq-tool-use:latest", temperature=0)

In [25]:
tools = [abc]

llm_with_tools = llm.bind_tools(tools)

In [26]:
response = llm_with_tools.invoke("Compute abc between 2 and 3")

In [28]:
response.tool_calls

[{'name': 'abc',
  'args': {'num1': 2, 'num2': 3},
  'id': 'cedbf9e6-24f1-4140-ba7a-e01fd877b18c',
  'type': 'tool_call'}]